In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler

# Set the random seed for reproducibility
np.random.seed(42)

#Set Visualization style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("deep")
plt.rcParams['figure.figsize'] = [12, 7]


#### 1. Generate synthetic telecom network kpi data

In [ ]:
def generate_telecom_kpi_data(days=14, samples_per_day=96):
    """
       Generate synthetic telecom network KPI data.
    
        Parameters:
        - days: Number of days to generate data for
        - samples_per_day: Number of samples per day (96 = every 15 minutes)
        
        Returns:
        - DataFrame with timestamp and KPI columns
    """
    total_samples = days * samples_per_day
    start_date = datetime(2024, 2, 1)
    timestamps = [start_date + timedelta(minutes=15*i) for i in range(total_samples) ]
    
    # Base Data patterns
    time_indices = np.arange(total_samples)
    
    
    